In [ ]:
import os
import numpy as np
import re
from diffaux.validation.read_size_validation_data import read_size_data, validation_info
from diffaux.validation.read_size_validation_data import get_author_list

In [ ]:
authors_rm = get_author_list(validation_info["Re_vs_Mstar2"])

In [ ]:
Rmdata = read_size_data({}, authors_rm, info_key="Re_vs_Mstar2")

## Plot Behavior of Fit Coefficients for Median Sizes

In [ ]:
notebook_dir = "/Users/kovacs/cosmology/diff_notebooks"
from diffaux.validation.plot_size_data import plot_size_data
from diffaux.validation.plot_size_data import PLOT_DRN

In [ ]:
plot_size_data(
    Rmdata,
    validation_info,
    authors_rm,
    info_keys=["Re_vs_Mstar2"],
    summary_only=True,
    plotdir=os.path.join(notebook_dir, PLOT_DRN),
)

## Setup Choices for Fitting the Behavior of Median Sizes with M*

In [ ]:
from diffaux.size_modeling.fit_size_data import collect_data_vectors, Samples_MFit2, Samples_MFit

data_vectors = collect_data_vectors(
    Rmdata, Samples_MFit2, validation_info, fit_type="Re_vs_Mstar2", lambda_min=0.5, lambda_max=1.0
)
print(data_vectors)
from diffaux.size_modeling.fit_size_data import Names_MFit2

print(Names_MFit2)

In [ ]:
from diffaux.size_modeling.fit_size_data import MFit2Params_Sig_initial, MFit2Params_Lin_initial

print(MFit2Params_Lin_initial)

In [ ]:
from diffaux.size_modeling.fit_size_data import assign_p0_values_to_fits, _linear

# assign initial values to fits dict and plot
fits = assign_p0_values_to_fits(MFit2Params_Lin_initial, fit_type="Re_vs_Mstar2")
print(fits)

In [ ]:
from diffaux.validation.plot_size_fits import plot_fits

fit_type = "Re_vs_Mstar2"
plot_fits(
    fits[fit_type],
    data_vectors[fit_type],
    Samples_MFit2,
    validation_info[fit_type],
    plotdir=os.path.join(notebook_dir, PLOT_DRN),
    title="InitialGuess",
    pltname="FitRe_vs_Mstar2_Lin_{}.png",
    fit_label="z",
    func=_linear,
)

## Fit the Coefficients and Save Results

In [ ]:
fit_type = "Re_vs_Mstar2"
from diffaux.size_modeling.fit_size_data import fit_parameters, _sigmoid, _linear

fits = {}
from diffaux.size_modeling.fit_size_data import Xvalue_MFit2

fits[fit_type] = fit_parameters(data_vectors[fit_type], Xvalue_MFit2, MFit2Params_Lin_initial, func=_linear)
print(fits["Re_vs_Mstar2"])

In [ ]:
plot_fits(
    fits[fit_type],
    data_vectors[fit_type],
    Samples_MFit2,
    validation_info[fit_type],
    func=_linear,
    data=Rmdata[fit_type],
    authors=authors_rm,
    plotdir=os.path.join(notebook_dir, PLOT_DRN),
    pltname="FitRe_vs_Mstar2_{}.png",
    fit_label="z",
    title="Linear_bestfit",
)

In [ ]:
import pickle
from diffaux.size_modeling.fit_size_data import MFit2Parameters, MFitParameters
from diffaux.size_modeling.fit_size_data import write_fit_parameters, read_fit_parameters

In [ ]:
write_fit_parameters(fits, fitdir="./", fit_type="Re_vs_Mstar2")

## Read in Fit Parameters and Generate Sizes

In [ ]:
fit_pars_Mfit2, _ = read_fit_parameters(MFit2Parameters, fit_type="Re_vs_Mstar2")
print(fit_pars_Mfit2)

In [ ]:
from diffaux.size_modeling.fit_size_data import assemble_mixed_fit_parameters

fit_pars = assemble_mixed_fit_parameters(Samples_MFit)
print(fit_pars)

In [ ]:
# try to tinker the beta coefficients for Quiescent galaxies
print(fit_pars["Quiescent"].beta_Quiescent)
fit_pars_test = {}
fit_pars_tmp = {}
fit_pars_test["Starforming"] = fit_pars["Starforming"]
fit_pars_tmp["Quiescent"] = fit_pars["Quiescent"]._replace(beta_Quiescent=np.asarray([0.71, 0.055]))
fit_pars_test["Quiescent"] = fit_pars_tmp["Quiescent"]._replace(logMp_Quiescent=np.asarray([10.0, 0.4]))
print(fit_pars_test["Quiescent"].beta_Quiescent)
print(fit_pars_test["Quiescent"].logMp_Quiescent)

In [ ]:
# tinker with dict to save values
print(fits["Re_vs_Mstar2"]["beta_Quiescent"])
fits["Re_vs_Mstar2"]["beta_Quiescent"]["popt"] = np.asarray([0.71, 0.055])
fits["Re_vs_Mstar2"]["logMp_Quiescent"]["popt"] = np.asarray([10.0, 0.4])
print(fits["Re_vs_Mstar2"]["beta_Quiescent"], fits["Re_vs_Mstar2"]["logMp_Quiescent"])

In [ ]:
# Write tinkered fir coefficients to pkl file
write_fit_parameters(fits, fitdir="./", fn="{}_betaQ_adjusted_fit_parameters.pkl", fit_type="Re_vs_Mstar2")

In [ ]:
fit_pars_betaQ, _ = read_fit_parameters(
    MFit2Parameters, fit_type="Re_vs_Mstar2", fn="{}_betaQ_adjusted_fit_parameters.pkl"
)
from diffaux.size_modeling.fit_size_data import DEFAULT_MIXED_FIT_FILENAMES

MODQ_MIXED_FIT_FILENAMES = DEFAULT_MIXED_FIT_FILENAMES.copy()
MODQ_MIXED_FIT_FILENAMES["Quiescent"] = "{}_betaQ_adjusted_fit_parameters.pkl".format(fit_type)
fit_pars_Qmod = assemble_mixed_fit_parameters(Samples_MFit, plknames=MODQ_MIXED_FIT_FILENAMES)
print(fit_pars_Qmod)

In [ ]:
# generate sizes
import numpy as np
from diffaux.size_modeling.fit_size_data import (
    generate_sizes,
    get_color_mask,
    _linear,
    median_size_vs_Mstar2,
    get_median_sizes,
)
from diffaux.size_modeling.fit_size_data import (
    DEFAULT_FIT_FUNCTIONS,
    DEFAULT_SIZE_FUNCTIONS,
    DEFAULT_MIXED_FIT_FITTYPES,
)

N = 6000
lM_lo = 7.5
lM_hi = 12.0
z_lo = 0.0
z_hi = 3.0
UVJcolor_cut = 1.5
log_Mstar = np.random.uniform(low=lM_lo, high=lM_hi, size=N)
redshift = np.random.uniform(low=z_lo, high=z_hi, size=N)
color_gal = np.random.uniform(low=-0.2, high=2.3, size=N)

In [ ]:
# check how R_median changes with different fit parameters
R_median = {}
keys = ["Mfit2 Parameters", "Q-modified Parameters"]
pars = [fit_pars, fit_pars_Qmod]

for k, p in zip(keys, pars):
    R_median[k] = get_median_sizes(
        p,
        log_Mstar,
        redshift,
        color_gal,
        N,
        Samples_MFit2,
        fit_types=DEFAULT_MIXED_FIT_FITTYPES,
        UVJcolor_cut=UVJcolor_cut,
        fit_funcs=DEFAULT_FIT_FUNCTIONS,
        size_funcs=DEFAULT_SIZE_FUNCTIONS,
    )
    print(np.min(R_median[k]), np.max(R_median[k]))

In [ ]:
from diffaux.validation.plot_utilities import get_nrow_ncol, save_fig
import matplotlib.pyplot as plt
from scipy.stats import binned_statistic


def plot_R_median(
    R_median,
    color_gal,
    log_Mstar,
    redshift,
    val_info,
    sample="Quiesent",
    z_lo=0.0,
    z_hi=3.0,
    Nz=7,
    logM_lo=7.5,
    logM_hi=12.0,
    NM=6,
    plotdir=PLOT_DRN,
    xtitle="parameter_tests",
    pltname="R_median_vs_Mstar_zbins_{}.png",
):
    z_bins = np.linspace(z_lo, z_hi, Nz)
    nrow, ncol = get_nrow_ncol(len(R_median.keys()))

    fig, ax_all = plt.subplots(nrow, ncol, figsize=(ncol * 7, nrow * 5), sharey=True)
    for ax, (k, v) in zip(ax_all.flat, R_median.items()):
        for z_min, z_max in zip(z_bins[0:-1], z_bins[1:]):
            zmask = (redshift >= z_min) & (redshift < z_max)
            cmask = get_color_mask(color_gal, sample)
            mask = zmask & cmask
            ztitle = f"${z_min:.1f} \\leq z < {z_max:.1f}$"
            result = binned_statistic(
                log_Mstar[mask],
                [v[mask], v[mask] ** 2],
                bins=NM,
                range=(logM_lo, logM_hi),
                statistic="mean",
            )
            means, means2 = result.statistic
            bin_edges = result.bin_edges
            bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2.0
            ax.plot(bin_centers, means, label=f"{ztitle}", linewidth=3)

        ax.legend(loc="best", title=k)
        ax.set_yscale("log")
        ax.set_xlabel(val_info["xlabel"])
        ax.set_ylabel(val_info["ylabel"])

    fig.suptitle(f"{sample} Median Sizes", y=0.92, fontweight="bold")
    plotdir = os.path.join(notebook_dir, PLOT_DRN)
    save_fig(fig, plotdir, pltname.format(xtitle))

    return

In [ ]:
plot_R_median(R_median, color_gal, log_Mstar, redshift, validation_info["Re_vs_Mstar_data"])

In [ ]:
R_med = {}
Re = {}
keys = ["Mfit2 Parameters", "Q-modified Parameters"]
pars = [fit_pars, fit_pars_Qmod]

for k, p in zip(keys, pars):
    Re[k], R_med[k] = generate_sizes(
        p,
        log_Mstar,
        redshift,
        color_gal,
        fit_types=DEFAULT_MIXED_FIT_FITTYPES,
        samples=Samples_MFit2,
        size_funcs=DEFAULT_SIZE_FUNCTIONS,
        fit_funcs=DEFAULT_FIT_FUNCTIONS,
        scatter=0.2,
    )
    print(np.min(Re[k]), np.max(Re[k]))

## Plot Results and Compare Generated Sizes with Validation Data

In [ ]:
authors_re = get_author_list(validation_info["Re_vs_z_data"])
Redata = read_size_data({}, authors_re, info_key="Re_vs_z_data")
authors_rd = get_author_list(validation_info["Re_vs_Mstar_data"])
Rddata = {}
for author in authors_rd:
    samples = validation_info["Re_vs_Mstar_data"][author]["samples"]
    for sample in samples:
        if "All" not in sample:
            Rddata = read_size_data(Rddata, authors_rd, info_key="Re_vs_Mstar_data", sample=sample)
tests = [Redata, Rddata]

In [ ]:
# make validation plots using data for measured sizes and median sizes
from diffaux.validation.plot_size_fits import plot_generated_sizes

for k in Re.keys():
    for test, ikey in zip(tests, ["Re_vs_z_data", "Re_vs_Mstar_data"]):
        authors = get_author_list(validation_info[ikey])
        plot_generated_sizes(
            Re[k],
            R_med[k],
            color_gal,
            log_Mstar,
            redshift,
            Samples_MFit2,
            authors,
            test[ikey],
            validation_info[ikey],
            logM_lo=7.5,
            plotdir=os.path.join(notebook_dir, PLOT_DRN),
            pltname="GalaxySizes_vs_Mstar_zbins_MixedFit_{}_{{}}.png".format(re.sub(" ", "_", k)),
        )

In [ ]:
# test the test
from diffaux.size_modeling.tests import test_generate_sizes

types = ("SF", "Q")
var = "R_med_{}"
vals = [var.format(t) for t in list(types)]

In [ ]:
from importlib.resources import files

TESTDATA_DRN = files("diffaux").joinpath("size_modeling/tests/testing_data")
# TESTDATA_DRN = "./"

import numpy as np

from diffaux.size_modeling.fit_size_data import (
    Samples_MFit,
    generate_sizes,
    assemble_mixed_fit_parameters,
    DEFAULT_MIXED_FIT_FITTYPES,
    DEFAULT_SIZE_FUNCTIONS,
    DEFAULT_FIT_FUNCTIONS,
)


def chk_generate_sizes(
    lM_lo=7.5,
    lM_hi=12.0,
    Nm=10,
    z_lo=0.0,
    z_hi=3.0,
    Nz=4,
    color_SF=0.0,
    color_Q=2,
    testdata_dir=TESTDATA_DRN,
    UVJcolor_cut=1.5,
    scatter=0.2,
    read=True,
    fn="generate_sizes_test_data.txt",
    rtol=1e-4,
    variables=("logM", "z"),
    types=("SF", "Q"),
    var="R_med_{}",
):
    vals = [var.format(t) for t in list(types)]
    header = list(variables) + vals
    filename = os.path.join(testdata_dir, fn)
    if read:
        usecols = [header.index(header[i]) for i in range(len(header))]
        data = np.loadtxt(filename, unpack=True, usecols=usecols, skiprows=1)
        test_data = dict(zip(header, data))
        logM = test_data["logM"]
        z = test_data["z"]
    else:
        logM_values = np.linspace(lM_lo, lM_hi, Nm)
        print(logM_values)
        z_values = np.linspace(z_lo, z_hi, Nz)
        z = np.repeat(z_values, Nm)
        logM = np.tile(logM_values, Nz)

    # assemble fit parameters and generate new size data
    results = {"logM": logM, "z": z}
    fit_pars = assemble_mixed_fit_parameters(Samples_MFit)
    for color, t in zip([color_SF, color_Q], types):
        _res = generate_sizes(
            fit_pars,
            logM,
            z,
            np.repeat(color, Nm * Nz),
            fit_types=DEFAULT_MIXED_FIT_FITTYPES,
            samples=Samples_MFit,
            size_funcs=DEFAULT_SIZE_FUNCTIONS,
            fit_funcs=DEFAULT_FIT_FUNCTIONS,
            UVJcolor_cut=UVJcolor_cut,
            scatter=scatter,
        )

        results[var.format(t)] = _res[1]

    # test or save median values
    if read:
        for k in header[len(variables) :]:
            test = np.isclose(results[k], test_data[k], rtol=rtol)
            assert np.all(test)
    else:
        np.savetxt(
            filename, np.vstack(list(results[k] for k in results)).T, fmt="%11.4e", header="    ".join(header)
        )
        print(f"Writing test data in {filename}")

    return

In [ ]:
chk_generate_sizes(read=False)

In [ ]:
chk_generate_sizes()

In [ ]:
test_generate_sizes()